# Import Libraries

In [152]:

from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType,StructField,IntegerType
from pyspark.sql.functions import stddev,mean,avg,lit
import numpy as np
from pyspark.sql.functions import round




# Sampled data-file paths

In [2]:
# file paths
business_file="Sample_Datasets/montreal_business/part-00000-b5f251d0-79e6-47a8-a405-042eb7b7894e-c000.snappy.parquet"
reviews_file="Sample_Datasets/montreal_reviews/part-00000-f0e4463e-0ac9-402e-b995-734cbefc958e-c000.snappy.parquet"
users_file="Sample_Datasets/montreal_users/part-00000-a7d49d78-89a7-478f-a577-0efe02dca047-c000.snappy.parquet"


# Initialize spark session

In [3]:
app_name="Collaborative filtering for restaurant recommendation"

def init_spark():
    spark = SparkSession \
        .builder \
        .appName(app_name) \
        .getOrCreate()
    return spark


# Load Dataset in Apache Spark

In [4]:
spark=init_spark()
business_df = spark.read.parquet(business_file)
reviews_df=spark.read.parquet(reviews_file)
users_df=spark.read.parquet(users_file)

## Selecting required features

In our project, we are only concerned with a subset of columns from the dataset, specifically those that are relevant to our goal of recommending restaurants in Montreal. Therefore, we extract the necessary features from the business_df table, including the id, name, stars, category. 
Similarly, we filter the reviews_df table to include only reviews for the selected restaurants by performing an inner join with business_df.

In [5]:
business_df = business_df.select("business_id","name", "stars", 
                                 "review_count", "address", "city", "state", "postal_code", "longitude", 
                                 "categories", "latitude").withColumnRenamed("stars", "stars_restaurant")
reviews_df = reviews_df.join(business_df, on='business_id', how='inner')

## Preparing Data for ALS: Convert String to index
Prior to initiating the modeling process, it is essential to transform all the relevant columns to integer type for compatibility with the ALS model from pyspark. The columns requiring conversion are the business_id and user_id. We accomplish this by leveraging the StringIndexer function, which we imported from pyspark.ml.feature.

In [6]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['business_id', 'user_id']]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(reviews_df).transform(reviews_df)
transformed.select(['business_id', 'user_id','business_id_index', 'user_id_index','stars','categories'])


DataFrame[business_id: string, user_id: string, business_id_index: double, user_id_index: double, stars: double, categories: string]

## Spliting the dataset into training and testing subsets 

Setting a seed value as 3 to make randomsplit output deterministic

In [7]:
(training, test) = transformed.randomSplit([0.8, 0.2],3)

In [8]:
# Compute the global mean
average = training.select('user_id','stars').withColumn("user_id",lit(1)).groupBy('user_id').mean()
global_average = average.select('avg(stars)').collect()[0][0]

print("Global Average:",str(global_average))
global_mean = training.agg(avg('stars')).collect()[0][0]
print("Global Average:",str(global_mean))


Global Average: 3.8922836512555232
Global Average: 3.8922836512555232


# Prediction using global average for comparison

In [9]:
#Evaluate the root mean squared error if we use the global average as a prediction for all rating, Our model should perform better than this


test_avg = test.withColumn('prediction',lit(global_mean))
    
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",predictionCol="prediction")
rmse = evaluator.evaluate(test_avg)

print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 1.2260849010325352


## Basic ALS model
The Apache Spark library provides various parameters for the ALS (Alternating Least Squares) algorithm, including:

- rank: the number of latent factors used in the model (default value: 10).
- maxIter: the maximum number of iterations to run (default value: 10).
- regParam: the regularization parameter used in ALS (default value: 1.0).
- implicitPrefs: a boolean value that indicates whether to use the explicit feedback ALS variant or the one adapted for implicit feedback data (default value: false, which means using explicit feedback).
- alpha: a parameter that applies to the implicit feedback variant of ALS, determining the baseline confidence in preference observations (default value: 1.0).
- nonnegative: a boolean value that specifies whether to use nonnegative constraints for least squares (default value: false).

In [10]:

#ALS model with default values
als=ALS(userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
als.setSeed(2)

ALS_e3f9c9cc18f8

In [11]:
#train the default model
model=als.fit(training)

In [12]:
# predict the stars for test set 
predictions=model.transform(test)

In [13]:
#evaludate default model
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

RMSE=1.3534108396501865


## Tuning Hyper Parameters

In [14]:
ranks=[5,30,50,100]
for rank in ranks:
    als = ALS(rank=rank,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
    als.setSeed(2)
    model = als.fit(training)
    evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
    predictions=model.transform(test)
    rmse=evaluator.evaluate(predictions)
    print("RMSE with latent factor "+str(rank) +" is="+str(rmse))
    

RMSE with latent factor 5 is=1.3524617852548415
RMSE with latent factor 30 is=1.3238220621391332
RMSE with latent factor 50 is=1.3182668564134439
RMSE with latent factor 100 is=1.309269792868218


In [15]:
epcohs=[10,15,20]
for epcoh in epcohs:
    als = ALS(rank=100,maxIter=epcoh,userCol="user_id_index",itemCol="business_id_index",ratingCol="stars",coldStartStrategy="drop",nonnegative=True)
    als.setSeed(2)
    model = als.fit(training)
    evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
    predictions=model.transform(test)
    rmse=evaluator.evaluate(predictions)
    print("RMSE with latent factor 100 and maxIter "+str(epcoh) +" is="+str(rmse))
    

RMSE with latent factor 100 and maxIter 10 is=1.309269792868218
RMSE with latent factor 100 and maxIter 15 is=1.2994357989444387
RMSE with latent factor 100 and maxIter 20 is=1.2990339815403404


# ALS Model With Bias

In [16]:
# Compute user mean and item mean on the training set
user_mean = training.groupBy('user_id_index').agg(avg('stars').alias('user_mean'))
item_mean = training.groupBy('business_id_index').agg(avg('stars').alias('item_mean'))
#remove bias from training set
interactions = training.join(user_mean, 'user_id_index').join(item_mean, 'business_id_index')
interactions = interactions.withColumn('user_item_interaction', col('stars') - col('user_mean') - col('item_mean') + global_mean)
interactions.select('stars','user_mean','item_mean','user_item_interaction').show(10)

+-----+-----------------+------------------+---------------------+
|stars|        user_mean|         item_mean|user_item_interaction|
+-----+-----------------+------------------+---------------------+
|  4.0|3.652173913043478| 4.133333333333334|  0.10677640487871143|
|  4.0|3.652173913043478| 3.989795918367347|  0.25031381984469814|
|  4.0|3.652173913043478|3.9105263157894736|  0.32958342242257155|
|  4.0|3.652173913043478| 4.235887096774194| 0.004222641437851227|
|  4.0|3.652173913043478| 3.925925925925926|  0.31418381228611914|
|  3.0|3.652173913043478| 4.244514106583072|   -1.004404368371027|
|  4.0|3.652173913043478|               4.0|  0.24010973821204518|
|  5.0|3.652173913043478| 4.042654028436019|   1.1974557097760266|
|  4.0|3.652173913043478|             3.125|   1.1151097382120452|
|  4.0|3.652173913043478|               4.0|  0.24010973821204518|
+-----+-----------------+------------------+---------------------+
only showing top 10 rows



In [17]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='stars', predictionCol='user_item_interaction_recomputed')

ranks=[5,10,30,50,100]
for rank in ranks:
    als = ALS(maxIter=20,rank=rank,userCol="user_id_index",itemCol="business_id_index",ratingCol="user_item_interaction",coldStartStrategy="drop",nonnegative=True)
    als.setSeed(2)
    model = als.fit(interactions)
    predictions=model.transform(test).withColumnRenamed('prediction','predicted_rating')
    predictions=  predictions.join(user_mean, 'user_id_index').join(item_mean, 'business_id_index')
    predictions = predictions.withColumn('user_item_interaction_recomputed', col('predicted_rating') + col('user_mean') + col('item_mean') - global_mean)
    rmse=evaluator.evaluate(predictions)
    print("RMSE with latent factor "+str(rank) +" is="+str(rmse))

RMSE with latent factor 5 is=1.187015001289878
RMSE with latent factor 10 is=1.1856944430863587
RMSE with latent factor 30 is=1.1831676942255571
RMSE with latent factor 50 is=1.1828233746778907
RMSE with latent factor 100 is=1.1813151492298182


# Final model 

In [18]:
als = ALS(maxIter=20,rank=100,userCol="user_id_index",itemCol="business_id_index",ratingCol="user_item_interaction",coldStartStrategy="drop",nonnegative=True)
als.setSeed(2)
model = als.fit(interactions)
predictions=model.transform(test).withColumnRenamed('prediction','predicted_rating')
predictions=  predictions.join(user_mean, 'user_id_index').join(item_mean, 'business_id_index')
predictions = predictions.withColumn('user_item_interaction_recomputed', col('predicted_rating') + col('user_mean') + col('item_mean') - global_mean)
  

# Evaluation

In [19]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='stars', predictionCol='user_item_interaction_recomputed')
rmse=evaluator.evaluate(predictions)
print("RMSE is="+str(rmse))

RMSE is=1.1813151492298204


In [20]:
std_dev = predictions.select(stddev('user_item_interaction_recomputed')).collect()[0][0]
print('Standard deviation of predictions:', std_dev)

std_dev_stars = predictions.select(stddev('stars')).collect()[0][0]
print('Standard deviation of stars:', std_dev_stars)

mean_prediction = predictions.select(mean('user_item_interaction_recomputed')).collect()[0][0]
print('Mean of predictions:', mean_prediction)

mean_stars = predictions.select(mean('stars')).collect()[0][0]
print('Mean of stars:', mean_stars)

print('Standard Deviation ratio ', std_dev/std_dev_stars)

predictions.select('business_id_index','stars','user_item_interaction_recomputed').withColumnRenamed('user_item_interaction_recomputed','predicted_rating').show(10)

Standard deviation of predictions: 0.9202296973985253
Standard deviation of stars: 1.1406615072644828
Mean of predictions: 3.973922158296197
Mean of stars: 3.8865761405816848
Standard Deviation ratio  0.8067508998400466
+-----------------+-----+------------------+
|business_id_index|stars|  predicted_rating|
+-----------------+-----+------------------+
|            207.0|  4.0| 4.476608654423227|
|            481.0|  3.0|3.9692444077048012|
|            133.0|  5.0| 4.186363026915593|
|           2089.0|  5.0| 5.002954443982572|
|            300.0|  5.0| 4.350132567377221|
|           3038.0|  5.0| 4.384552064231366|
|           1121.0|  5.0| 4.148446820071667|
|           1374.0|  4.0| 4.280622373214692|
|           1295.0|  5.0| 4.210098958015045|
|            149.0|  5.0|3.1068964112286785|
+-----------------+-----+------------------+
only showing top 10 rows



# Find Best Recommendations

In [160]:
def get_recommendations(target_user):
    """
    Returns top recommendations for a user.

    Returns
    -------
    :py:class:`pyspark.sql.DataFrame`
    a DataFrame of (itemCol, recommendations), where recommendations are
    stored as an array of ('name','business_id', 'stars', 'categories') Rows.
    """
    
    out = model.recommendForAllUsers(10).filter(col('user_id_index')==target_user).select("recommendations").take(10)
    min_rating = 1.0 
    max_rating = 5.0 
    topRestaurants = []
    ratings=[]
    user_mean_rating=(user_mean.filter(col('user_id_index')==30).take(1))[0]['user_mean']
    for item in out[0][0]:        
        topRestaurants.append(item.business_id_index)
        item_mean_rating=(item_mean.filter(col('business_id_index')==item.business_id_index).take(1))[0]['item_mean']
        ratings.append(item.rating+user_mean_rating+item_mean_rating-global_mean)

    normalized_ratings = [(rating - min_rating) / (max_rating - min_rating) for rating in ratings]
    scaled_ratings = [(normalized_rating * (max_rating - min_rating)) + min_rating for normalized_rating in normalized_ratings]
    final_ratings = [min(max(rating, min_rating), max_rating) for rating in scaled_ratings]
    
    restaurants = spark.createDataFrame(zip(topRestaurants,final_ratings)).toDF("business_id_index","ratings")

    return restaurants

In [165]:
def display_top10_recommendations(restaurants):
    """
    Displays the top 10 restaurant recommendations.
    """
    restaurants\
    .join(transformed, on = 'business_id_index', how = 'inner')\
    .select([ 'business_id_index','name', round('ratings',2), 'categories']).withColumnRenamed('round(ratings, 2)','rating')\
    .drop_duplicates(subset=['name']).sort(col('ratings'),ascending=False).show(10)


## Display the Top Recommendations

In [166]:
target_user = '85Jq6l707HWj2XpFw7qPGg'
user_id_index=(transformed.where(col('user_id')==target_user).select('user_id_index').take(1))[0]['user_id_index']
top10_recommendations = get_recommendations(user_id_index)
display_top10_recommendations(top10_recommendations)


+-----------------+--------------------+------+--------------------+
|business_id_index|                name|rating|          categories|
+-----------------+--------------------+------+--------------------+
|             2438|    Café Mucho Mucho|  4.51|Food, Coffee & Te...|
|             3841|Commission des Li...|  3.97|Restaurants, Danc...|
|             2777|Resto-Pub 100 Génies|  3.96|Burgers, Pizza, R...|
|              657|La Station des Sp...|   3.9|Restaurants, Burg...|
|              934|             Soubois|  3.88|Bars, Gastropubs,...|
|             1798|             Pandore|  3.77|Nightlife, Dance ...|
|             2699|           Lobby Bar|  3.76|Canadian (New), N...|
|             1337|    Restaurant Limon|  3.66|Mexican, Restaurants|
|             2217|            Échalote|  3.29|Pan Asian, Vietna...|
|             3518|      Resto-Bar-Code|  1.75|Canadian (New), R...|
+-----------------+--------------------+------+--------------------+

